# PHASE 2
Takes the data_bf.csv from PHASE1A/B and produces data_2a.csv andf data_2b.csv

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint


Here we import all relevant data.

In [3]:
covid19 = pd.read_csv('2_DATA/us-counties.csv')
unemployment = pd.read_excel('2_DATA/Unemployment.xls', sheet_name=0, skiprows=(0,1,2,3))
pov_est = pd.read_excel('2_DATA/PovertyEstimates.xls', sheet_name=0, skiprows=(0,1,2,3))
pop_data = pd.read_csv('2_DATA/co-est2019-alldata.csv', encoding ="ISO-8859-1")
pop_dens = pd.read_csv('2_DATA/census_county_interpolated.csv')

This cell below refines many of our large data sets into the few columns used as features

In [4]:
unemployment = unemployment[['FIPS', 'Unemployment_rate_2018', 'Median_Household_Income_2018', 'Med_HH_Income_Percent_of_State_Total_2018']]
poverty = pov_est[['FIPStxt', 'PCTPOVALL_2018']]
pop_dens = pop_dens[pop_dens['year'] == 2016]
density = pop_dens[['fips', 'popdensity']]
race = pop_dens[['fips', 'pct_blk', 'pct_asian', 'pct_native', 'pct_white']]

This cell defines the make fips function which takes state and county codes and creates an FIPS code

In [5]:
def make_fips(state, county):
  if (state < 10):
    state_temp = "0"+ str(state)
  else: 
    state_temp = str(state)
  if (county < 10):
    county_temp = "00" + str(county)
  elif (county <= 99):
    county_temp = "0" + str(county)
  else:
    county_temp = str(county)
  temp = state_temp + county_temp
  return int(temp)

In [6]:
pop_data['FIPS'] = [make_fips(int(row['STATE']), int(row['COUNTY'])) for index, row in pop_data.iterrows()]
population = pop_data[['FIPS', 'POPESTIMATE2019', 'RBIRTH2019', 'RDEATH2019']]

In [7]:
features = pd.merge(population, poverty, left_on='FIPS', right_on='FIPStxt')
features.head()
features = features.drop(['FIPStxt'], axis=1)
features.head()
features = pd.merge(features, unemployment, on='FIPS')
features.head()
features = pd.merge(features, density, left_on='FIPS', right_on='fips')
features.head()
features = pd.merge(features, race, on='fips')
features = features.drop(['fips'], axis=1)


In [8]:
unique_fips = covid19['fips'].unique()

#  Dealing with data from 1A

In [9]:
bf_1a_data = pd.read_csv('2_DATA/data_1a_bf.csv')
bf_1a_data = bf_1a_data.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'POPESTIMATE2019'], axis=1)
bf_1a_data = bf_1a_data[bf_1a_data['f'] > 0]
bf_1a_data.head()

,FIPS,beta,f
0,1073.0,3.582561e-07,0.088171
1,1081.0,1.353718e-06,0.117474
2,1097.0,6.400933e-07,0.113277
3,4005.0,1.660785e-06,0.183262
4,4013.0,4.999949e-01,0.000007


EXPORT:

In [10]:
master_data = pd.merge(features, bf_1a_data, on='FIPS')
master_data.to_csv('2_DATA/master_data_A.csv')

# Dealing with data from 1B

In [12]:
bf_1b_data = pd.read_csv('2_DATA/data_1b_bf.csv')
bf_1b_data = bf_1b_data[bf_1b_data.cost < .0001]
bf_1b_data = bf_1b_data.drop(['Unnamed: 0', 'POPESTIMATE2019'], axis=1)
bf_1b_data = bf_1b_data[bf_1b_data['f'] > 0]

bf_1b_data

,FIPS,beta,f,cost
2,1007.0,6.365119e-06,0.000068,1.167641e-08
3,1009.0,5.777033e-07,0.002578,4.014245e-06
4,1011.0,7.401973e-06,0.000032,1.152826e-09
5,1013.0,2.200600e-06,0.000090,5.737434e-09
8,1019.0,1.177406e-05,0.000032,6.295205e-07
...,...,...,...,...
2191,56033.0,4.064746e-06,0.000946,1.516708e-05
2192,56035.0,9.304787e-08,0.000038,4.734482e-10
2193,56037.0,4.988492e-08,0.003144,3.877834e-06
2194,56039.0,5.710729e-06,0.000072,5.694819e-08


EXPORT:

In [13]:
master_data = pd.merge(features, bf_1b_data, on='FIPS')
master_data = master_data.drop(['cost'], axis = 1)
master_data.to_csv('2_DATA/master_data_B.csv')